# 3: Text-Based Similarity Search using Amazon Bedrock Embeddings and FAISS in AWS

- SageMaker Notebook Kernel: `conda_python3`
- SageMaker Notebook Instance Type: ml.m5d.large | ml.t3.large

In this notebook, you'll explore the process of implementing a text-based similarity search engine using [Amazon Bedrock](https://aws.amazon.com/bedrock/) Embeddings, [LangChain](https://python.langchain.com/), and the [Faiss](https://ai.meta.com/tools/faiss) (Facebook AI Similarity Search) library. The notebook focuses on loading text data from Amazon Bedrock documentation, preprocessing the content, and using Titan embeddings to convert text into embeddings. We then leverage FAISS to create an efficient similarity search database. The notebook aims to demonstrate the end-to-end flow of scraping, preprocessing, vectorizing, and performing similarity queries, using AWS services.

![](./assets/images/vector-store-workflow.png)

## Runtime 

This notebook takes approximately 10 minutes to run.

## Contents

1. [Prerequisites](#prerequisites)
1. [Setup](#setup)
1. [Load and prepare documents](#load-and-prepare-documents)
1. [Load documents into FAISS and run similarity search](#load-documents-into-faiss-and-run-similarity-search)

## Prerequisites

`amazon.titan-embed-text-v1` enabled in the Amazon Bedrock console in `us-west-2`


## Setup

Let's start by installing and importing the required packages for this notebook. 

<div class="alert alert-block alert-warning"><b>Note:</b> Verify that the notebook kernel is `conda_python3`. Also, if you run into an issue where a module can't be imported after installation, restart the notebook kernel, then rerun the import notebook cell.</div>

In [ ]:
%pip install requests --quiet
%pip install beautifulsoup4 --quiet
%pip install markdownify --quiet
%pip install unstructured --quiet
%pip install tqdm --quiet # this is the progress bar component https://github.com/tqdm/tqdm
%pip install langchain==0.0.317 --quiet
%pip install nltk --quiet
%pip install "faiss-cpu>=1,<2" --quiet

In [ ]:
import boto3
import json
import nltk

from IPython.display import display, IFrame
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import BedrockEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, NLTKTextSplitter
from langchain.vectorstores import FAISS

from lib.aws_docs_url_loader import AWSDocsURLLoader

***

Next, initialize the Amazon Bedrock boto3 client. The embeddings model we will use is `amazon.titan-embed-text-v1`. 

LangChain is a framework for developing applications powered by language models. The library provides LLM model adapters, data retrieval components, text splitters, conversation memory and storage, as well as components that wire all of these things together. It also provides agents which orchestrate LLMs and tools that go beyond chat interfaces. You will dive deeper into LangChain in the following notebooks, but for this one, you'll only use the BedrockEmbeddings and FAISS vector store helper classes.

- BedrockEmbeddings is an adapter class around the boto3 Bedrock client
- FAISS is an adapter class around the FAISS cpu library from Facebook

***

In [ ]:
bedrock_client = boto3.client("bedrock-runtime", region_name="us-west-2")

embeddings_model_id = "amazon.titan-embed-text-v1"

bedrock_embeddings = BedrockEmbeddings(client=bedrock_client, model_id=embeddings_model_id)

## Load and prepare documents

The `AWSDocsUrlLoader` class was built to cleanly load Amazon user guide documentation sources. Load the [what-is-bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html) page from the Bedrock user guide.

In [ ]:
urls = ["https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html"]

url_loader = AWSDocsURLLoader(urls=urls, mode="md", show_progress_bar=True, continue_on_failure=True)

documents = url_loader.load()

***

Take a look at the content produced by the loader.

***

In [ ]:
margin="\n\n\n"
line_spacer = f"{margin}{'='*75}{margin}"

print(line_spacer.join([doc.page_content for doc in documents[0:2]]))

***

Next, you will split the texts into smaller chunks. But first, we need to talk about tokens. Each word of text is split up into one or more tokens by a tokenizer before it's passed to the model. This is the case for both transformer based embedding and llm models. One common method for tokenization is `Byte-Pair Encoding`, where the most frequently occurring pairs of characters or bytes are merged into a single token until a certain number of tokens or vocabulary size is reached. This allows for a more compact representation of text while still allowing for the model to handle unseen words. A helpful run of thumb is that one token generally corresponds to ~4 characters of text for common english. Let's see how the tokenizer separates `What is Amazon Bedrock?` into tokens. Notice how Bedrock was split into two tokens.

![](./assets/images/what-is-amazon-bedrock-tokenized.png)

Transformer models have a fixed number of input dimensions and a fixed number of output dimensions. For the embeddings model we are using, `amazon.titan-embed-text-v1`, the input size is 8192 dimensions (tokens) and the output size is 1536 dimensions. When we pass a smaller number of input tokens, the remaining space gets padded but the output size remains the same. So, whether or not we pass in 1 token or 8192 tokens we are going to get an output of 1536 dimensions. You can understand that this is going to be an issue when trying to compare the vectors of those two extreme cases for similarity.

So, to get the best results from the search, we want to be consistent in the number of tokens we use for each embedding. An exact method is to tokenize the text, then split the tokens into chunks based on a fixed token count, then convert the tokens back into text, which will create equal length embeddings. But, we don't have access to the tokenizer for the Titan embeddings model, so we will use a text based method instead. The are many different text based methods, but for this workshop we will use the [NLTK](https://www.nltk.org/) text splitter provided by LangChain. _For more information on the methods, see [document transformers](https://python.langchain.com/docs/modules/data_connection/document_transformers/)_. The `NLTKTextSplitter` tries to keep sentences together preferring not to split in the middle of a sentence which can cause boundary issues. Since we are using the embeddings to find relevant information for a retrieval augmented chatbot, we want enough text to be meaningful, but not too much text to loose relevance. In addition, we will be asking questions of this data which tend to be short and that can affect the similarity score if the embeddings are generated with lengthy passages. Determining, the perfect method, chunk size, and overlap depends on the problem you're trying to solve, so it's recommended to test with various methods and configurations.

For this workshop, you'll use a chunk size of 1000 and an overlap of 50, so go ahead and run the code below andd take a look at the first result.

***

In [ ]:
nltk.download("punkt")

text_splitter = NLTKTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)
print(f"Document chunk count: {len(split_documents)}")
split_documents[0]

***

Next, you'll iterate over the split texts and print out the contents to see what each chunk looks like.

***

In [ ]:
print(line_spacer.join([doc.page_content for doc in split_documents[0:5]]))

## Load documents into Faiss and run similarity search

Faiss is a library for efficient similarity search and clustering of dense vectors, and you can use it to run similarity searches in memory with the downloaded data. LangChain provides a helper class for Faiss with a static method, `from_documents`. This method iterates over the split texts and and calls the Bedrock Embeddings endpoint to create the embeddings and inserts them into the database. Then it returns an instance of itself where we can run similarity searches.

In [ ]:
db = FAISS.from_documents(split_documents, bedrock_embeddings)

***

Let's define a helper method to run the similarity search and print out the returned documents with the similarity score.

***

In [ ]:
def similarity_search(text, k=2):
    results = db.similarity_search_with_score(text, k)
    print(
        line_spacer.join(
            f"[{index + 1}. score = {score}] \n\n {r.page_content} \n\n {str(r.metadata)}"
            for index, (r, score) in enumerate(results)
        )
    )

***

Run a few searches to see what's returned. You can visit the page [what-is-bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html) to ask a question of your own.

***

In [ ]:
similarity_search("What is Amazon Bedrock?")

In [ ]:
similarity_search("What large language models are available on bedrock?")

In [ ]:
similarity_search("<insert question here>")

## Notebook complete

With Faiss, it's easy to test embeddings with similarity searches locally, but for our application we will want a more robust and scalable solution. Move to the next notebook to load the entire Amazon Bedrock user guide into Amazon OpenSearchServerless.
